# 🌰 [Day 15](https://adventofcode.com/2019/day/15)

In [1]:
def run_program(p, inputs, init_op=0, init_base=0):
    #Inputs are given in reverse order (pop)
    op = init_op
    relative_base = init_base
    last_diagnostic = []
    while p[op] != 99:
        codes = "%05d" % p[op]
        codes = [int(codes[0]), int(codes[1]), int(codes[2]), int(codes[3:])]
        # inputs
        if codes[-1] == 3:
            if not len(inputs):
                # Game is waiting for next move
                break
            assert codes[1] == 0
            p[p[op + 1] + (relative_base if codes[2] == 2 else 0)] = inputs.pop()
            op += 2
        # unary ops
        elif codes[-1] in  [4, 9]:
            # read parameter
            assert codes[1] == 0
            param = p[op + 1]
            if (codes[2] % 2) == 0:
                try:
                    param = p[param + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    param = 0
            # output
            if codes[-1] == 4:
                last_diagnostic.append(param)
            # update relative base
            else:
                relative_base += param
            # next instr
            op += 2
        else:
            # read parameters in correct mode
            x, y = p[op + 1:op + 3]
            if (codes[2] % 2) == 0:
                try:
                    x = p[x + (relative_base if codes[2] == 2 else 0)]
                except IndexError:
                    x = 0
            if not (codes[1] % 2):
                try:
                    y = p[y + (relative_base if codes[1] == 2 else 0)]
                except IndexError:
                    y = 0
            # Read target and allocate more memory if needed
            target = p[op + 3] + (relative_base if codes[0] == 2 else 0)
            if target >= len(p): 
                p += [0] * (target - len(p) + 1)
            # addition and multiplication
            if codes[-1] in [1, 2]:
                p[target] = x + y if codes[-1] == 1 else x * y
                op += 4
            # Comparison result
            elif codes[-1] == 7:
                p[target] = int(x < y)
                op += 4
            elif codes[-1] == 8:
                p[target] = int(x == y)
                op += 4
            # Jump if eq
            elif (codes[-1] == 5 and x != 0) or (codes[-1] == 6 and x == 0):
                op = y  
            # Jump instruction that failed their test
            else:
                op += 3
    return last_diagnostic, 0, op, relative_base


import heapq
import numpy as np
from collections import defaultdict

def robot_bfs(program, n=50):
    """Breadth-first search"""
    dists = defaultdict(lambda: float('inf'))
    # Mapout
    maze = np.zeros((n, n))
    i, j = n // 2, n // 2
    maze[i, j] = 1
    oxg = None, None
    # (cost, point, program state)
    queue = [(0, (i, j), program, 0, 0)]
    heapq.heapify(queue)
    min_cost = float('inf')
    
    # current state
    while len(queue):
        cost, coords, program_state, op, base = heapq.heappop(queue)
        for direction in [1, 2, 3, 4]:
            p = [x for x in program_state]
            out, _, next_op, next_base = run_program(p, [direction], op, base)
            out = out[0]
            # Hit a wall
            if out == 0: 
                continue
            # Advance
            next_coords = (coords[0] + (1 if direction == 4 else -1 if direction == 3 else 0), 
                           coords[1] + (1 if direction == 2 else -1 if direction == 1 else 0))
            if dists[next_coords] > cost + 1:
                dists[next_coords] = cost + 1
                if out == 1:
                    maze[next_coords[0], next_coords[1]] = 1
                    heapq.heappush(queue, (cost + 1, next_coords, p, next_op, next_base))
                elif out == 2: ## Minimum costs to reach oxygen
                    maze[next_coords[0], next_coords[1]] = 2
                    min_cost = min(min_cost, cost + 1)
                    oxg = next_coords
    return min_cost, maze, oxg

import matplotlib.pyplot as plt
import matplotlib.animation as animation
def oxygen_bfs(maze, oxg, verbose=True):
    queue = [(oxg[0], oxg[1], 0)]
    max_time = 0
    if verbose:
        fig = plt.figure()
        plt.axis('off')
        ims = []
    while len(queue):
        x, y, time = queue.pop(0)
        propagate = False
        for i, j in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
            if maze[i, j] == 1:
                maze[i, j] = 2
                propagate = True
                queue.append((i, j, time + 1))
                max_time = max(time + 1, max_time)
        time += int(propagate)
        if verbose:
            im = plt.imshow(maze)
            ims.append([im])
    
    if verbose:
        ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)
        ani.save('day15.mp4')
        plt.close(fig)
    return time

In [2]:
with open("inputs/day15.txt", 'r') as f:
    inputs = list(map(int, f.read().split(',')))
    min_cost, maze, oxg = robot_bfs(inputs)
    
print("The shortest path to oxygen takes {} steps".format(min_cost))

The shortest path to oxygen takes 216 steps


In [3]:
print("The oxygen spreads through the whole space in {} minutes".format(
    oxygen_bfs(maze, oxg)))
from IPython.display import Video
Video("day15.mp4")

The oxygen spreads through the whole space in 326 minutes
